In [1]:
import requests, json

In [2]:
from pysrc.mfp import MFPApi

In [3]:
api = MFPApi()

In [4]:
import datetime, dateutil, pytz
start = datetime.datetime(2013, 1, 1, tzinfo=pytz.timezone('US/Pacific'))
end = datetime.datetime(2020, 1, 1, tzinfo=pytz.timezone('US/Pacific'))
data = api.load_cals_range_async(start, end, 500)

Downloaded 2557 of 2557 (0 active threads)...         
Parsing 2557 of 2557                


In [5]:
%matplotlib
from matplotlib import pyplot as plt

Using matplotlib backend: TkAgg


In [32]:
data.sort(key=lambda t: t[0])
x,y = list(zip(*data))
plt.scatter(x,y, s=2)

In [6]:
def _deq(d1, d2):
    return d1.year == d2.year and d1.month == d2.month and d1.day == d2.day
def lookup(date):
    date = dateutil.parser.parse(date)
    for a,b in data:
        if _deq(a, date):
            return b
    

In [ ]:
lookup('2016-04-15')

In [ ]:
src = api.session.get("https://www.myfitnesspal.com/food/diary?date=2016-04-15").content

In [ ]:
import pyquery
Q = pyquery.PyQuery

In [ ]:
import lxml.etree as etree
for p in Q(api._wtf.encode('utf-8'))("#diary-table > tbody > .total"):
    print(p)

In [12]:
url = "https://www.myfitnesspal.com/reports/results/nutrition/Calories/%s.json?report_name=Calories"
s = api.session
days = 1800
rsp = s.get(url%days)
rsp.raise_for_status()
obj = rsp.json()

In [15]:
def parse_obj(obj, days):
    today = datetime.datetime.now()
    then = today - datetime.timedelta(days=days-1)
    day = datetime.timedelta(days=1)

    odata = obj['data']
    newdata = []
    first = 0
    for i, o in enumerate(odata):
        date = o['date']
        total = o['total']
    #     if total != 0 and first == 0:
    #         first = i
        daystr = "%s/%02d"%(then.month, then.day)
        if date != daystr:
            print("date mismatch: got '%s' expected '%s'"%(date, daystr))
        newdata.append((then, total))
        then += day
    return newdata
newdata = parse_obj(obj, days)
x2,y2 = list(zip(*newdata))
plt.close()
plt.scatter(x2,y2, s=2)

In [16]:
data.sort(key=lambda t: t[0])
x,y = list(zip(*data))
plt.scatter(x,y, s=2)

In [17]:
url = "https://www.myfitnesspal.com/reports/results/progress/1/%s.json?report_name=Weight"
s = api.session
days = 1800
rsp = s.get(url%days)
rsp.raise_for_status()
obj2 = rsp.json()

In [18]:
data.sort(key=lambda t: t[0])
x,y = list(zip(*data))
plt.close()
fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)
ax1.scatter(x,y, s=2)

In [19]:
weight = parse_obj(obj2, days)
x3,y3 = list(zip(*weight))
ax2 = ax1.twinx()
ax2.plot(x3,y3, color="orange")

In [71]:
import datetime, dateutil, pytz
start = datetime.datetime(2015, 2, 7)
end = datetime.datetime(2015, 8, 1)
data4 = api.load_cals_range_async(start, end, 100)

Downloaded 176 of 176 (0 active threads)...        
Parsing 176 of 176                


In [78]:
data5 = [(d, c) for d,c in data4 if c != 0]
data5.sort(key=lambda t: t[0])

In [79]:
sum(c for d,c in data5)/len(data5)

2107.1085714285714

In [80]:
fi = next(i for i, (d,w) in enumerate(weight) if _deq(start, d))
li = next(i for i, (d,w) in enumerate(weight) if _deq(end, d))

In [81]:
data6 = weight[fi:li]

In [82]:
data6[0][0], data6[-1][0]

(datetime.datetime(2015, 2, 7, 12, 16, 24, 733255),
 datetime.datetime(2015, 7, 31, 12, 16, 24, 733255))

In [83]:
data5[0][0], data5[-1][0]

(datetime.datetime(2015, 2, 7, 0, 0), datetime.datetime(2015, 8, 1, 0, 0))

In [84]:
wloss = data6[-1][1] - data6[0][1]

In [85]:
wloss

-17.799999999999983

In [86]:
wloss = abs(wloss)

In [87]:
cal_deficit = wloss * 3500

In [88]:
cal_eaten = len(data5)*2107.1

In [89]:
cal_deficit / len(data5)

355.99999999999966

In [90]:
cal_deficit / len(data5) + 2107.1

2463.0999999999995